# TRAINING

In [1]:
from entities import EntityTracker
from bow import BoW_encoder
import lstm_net
from embed import UtteranceEmbed
from actions import ActionTracker
from data_utils import Data

import util

In [2]:
import numpy as np

## Prepare Blocks

In [3]:
et = EntityTracker()
bow_enc = BoW_encoder()
emb = UtteranceEmbed()
at = ActionTracker(et)

## Prepare Data

In [5]:
trainset = Data(et, at).trainset
trainset[:10]

[('good morning', 3),
 ("i'd like to book a table with italian food", 7),
 ('<SILENCE>', 13),
 ('in paris', 6),
 ('for six people please', 14),
 ('in a cheap price range please', 9),
 ('<SILENCE>', 1),
 ('actually i would prefer for two people', 10),
 ('instead could it be in madrid', 10),
 ('instead could it be with spanish food', 10)]

## Define Network Parameters

In [6]:
obs_size = emb.dim + bow_enc.vocab_size + et.num_features
action_templates = at.get_action_templates()
action_size = at.action_size
nb_hidden = 128

## Define Network

In [7]:
import importlib
importlib.reload(lstm_net)
net = lstm_net.LSTM_net(obs_size=obs_size,
               action_size=action_size,
               nb_hidden=nb_hidden)

In [8]:
loss = 0
epochs = 10
for j in range(epochs):
    for i in range(len(trainset)):
        # get utterance
        u, r = trainset[i]
        # encode utterance
        u_ent = et.extract_entities(u)
        u_ent_features = et.context_features()
        u_emb = emb.encode(u)
        u_bow = bow_enc.encode(u)
        # concat features
        features = np.concatenate((u_ent_features, u_emb, u_bow), axis=0)
        # get action mask
        action_mask = at.action_mask()

        # forward propagation
        #  train step
        loss += net.train_step(features, r, action_mask)
        if i and i%6000 == 0:
            print(j, '.', i//6000, ':', loss/6000)
            loss = 0

0 . 1 : [ 1.24442208]
0 . 2 : [ 0.27273756]
0 . 3 : [ 0.17494497]
1 . 1 : [ 0.15699336]
1 . 2 : [ 0.1374225]
1 . 3 : [ 0.13203755]
2 . 1 : [ 0.13484542]
2 . 2 : [ 0.12210717]
2 . 3 : [ 0.12004256]
3 . 1 : [ 0.12504891]
3 . 2 : [ 0.11389817]
3 . 3 : [ 0.1131073]
4 . 1 : [ 0.11911698]
4 . 2 : [ 0.10826278]
4 . 3 : [ 0.10802703]
5 . 1 : [ 0.11429122]
5 . 2 : [ 0.10324478]
5 . 3 : [ 0.10344624]
6 . 1 : [ 0.11002142]
6 . 2 : [ 0.0982981]
6 . 3 : [ 0.09933174]
7 . 1 : [ 0.10634428]
7 . 2 : [ 0.09370458]
7 . 3 : [ 0.09550212]
8 . 1 : [ 0.10322022]
8 . 2 : [ 0.0893264]
8 . 3 : [ 0.0919035]
9 . 1 : [ 0.1004393]
9 . 2 : [ 0.08531643]
9 . 3 : [ 0.08909895]


In [17]:
u, r = trainset[307]

# encode utterance
u_ent = et.extract_entities(u)
u_ent_features = et.context_features()
u_emb = emb.encode(u)
u_bow = bow_enc.encode(u)

# concat features
features = np.concatenate((u_ent_features, u_emb, u_bow), axis=0)

# get action mask
action_mask = at.action_mask()

# forward
pred_template_id = net.forward(features, action_mask)
print(u, ':', action_templates[r], '||', action_templates[pred_template_id])

i love french food : which price range are looking for || api_call <cuisine> <location> <party_size> <rest_type>
